In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.mars_db
collection = db.items

In [4]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/melissa/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [5]:
# URL of page to be scraped
# This is not the given website, but I get returns, so I'm working with it for now
news_url = ("https://redplanetscience.com/#")
browser.visit(news_url)

In [6]:
# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [7]:
# Retrieve the latest element that contains news title and news_paragraph
title = soup.find('div', class_='content_title').text
article_teaser = soup.find('div', class_='article_teaser_body').text

# Display scrapped data 
print(title)
print(article_teaser)

# I think my issue was using find_all and .text together incorrectly and also not having the splinter set up.
# I'm still unsure about how it all relates and why it won't work without the other, but at least I can start moving forward

Mars 2020 Stands on Its Own Six Wheels
In time-lapse video, taken at JPL, captures the first time NASA's Mars 2020 rover carries its full weight on its legs and wheels.


In [8]:
# JPL Mars image

jpl_url = ("https://spaceimages-mars.com/")
browser.visit(jpl_url)

# Set up new instance of HTML object
html = browser.html

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

response = soup.find('a', class_='showimg')

# print(response)

href = response['href']

img_url = jpl_url + href

img_url


'https://spaceimages-mars.com/image/featured/mars2.jpg'

Mars Facts
https://galaxyfacts-mars.com/


Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.


Use Pandas to convert the data to a HTML table string.

In [9]:
# Let's try some table reading
# Look into weird character issues
facts_url = 'https://galaxyfacts-mars.com/'

tables = pd.read_html(facts_url)
tables

mars_facts = tables[0]
mars_facts

mars_html = mars_facts.to_html('mars_facts.html')
mars_html

Mars Hemispheres


Visit the astrogeology site here to obtain high resolution images for each of Mars's hemispheres.


You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.


Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.


Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [28]:
# Collect the image urls for the given website
# Ok, this works to pull one. How to get the other three?
hemisphere_url = 'https://marshemispheres.com/'
browser.visit(hemisphere_url)

# Do not forget new instance!!!!
html = browser.html

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

responses = soup.find_all('div', class_='item')
# responses

In [27]:
# Collect the image urls for the given website
# Why does it work with h3 and not h2?
hemisphere_url = 'https://marshemispheres.com/'
browser.visit(hemisphere_url)

# Do not forget new instance!!!!
html = browser.html

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Retreive all items that contain mars hemispheres information
images = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
image_urls = []

# Store the main_ul 
main_url = 'https://marshemispheres.com/'

# Loop through the items previously stored
for image in images: 
    
    # Grab link to image to set up nested browser.html 
    partial_img_url = image.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(main_url + partial_img_url)
    
    # HTML Object for site visited in nest 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every nested hemisphere information website 
    soup = BeautifulSoup(partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = main_url + soup.find('img', class_='wide-image')['src']
    
    title = soup.find('h2', class_='title').text
    
    # Append the retreived information into a list of dictionaries 
    image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [30]:
browser.quit()